In [2]:
# Packages
import pandas as pd
import datetime
from top2vec import Top2Vec


In [3]:
# Loading Data
df = pd.read_csv('data/combined_data.csv', index_col=0)
df

,creation_time,user,tweet,retweets,favorites,followers,lists
0,2022-09-26 20:14:17+00:00,GregL_Intel,I am looking forward to Intel Fellow @brendang...,10,54,4123,41
1,2022-09-26 20:11:39+00:00,GregL_Intel,Open software ecosystems are key to fostering ...,4,26,4123,41
2,2022-09-21 18:48:59+00:00,GregL_Intel,“AI Everywhere” will require optimized hardwar...,11,25,4123,41
3,2022-09-20 18:00:01+00:00,GregL_Intel,Our collaboration with @TU_Muenchen and the It...,2,12,4123,41
4,2022-09-19 22:52:59+00:00,GregL_Intel,"I am looking forward to having @AndrewYNg, fou...",19,85,4123,41
...,...,...,...,...,...,...,...
63028,2020-04-15 16:15:57+00:00,BarryJOGorman,@S_dF speaking of 'daily digital habits' - int...,2,1,909,103
63029,2020-04-15 13:54:53+00:00,BarryJOGorman,@nyike - if 'pushing envelope' - will always m...,1,0,909,103
63030,2021-02-03 18:17:58+00:00,RahulRJB,Even the mighty fall\n#FarmersProstest,1,2,47,0
63031,2022-10-06 06:42:42+00:00,jonsadventures,I've been writing notes for my second year non...,10,62,99,4


In [4]:
# Separating tweet text as text documents
docs = df.tweet.tolist()

# Some cleanup of words that are not removed by Top2Vec's stopword cleaning:
docs = [d.replace("https", "") for d in docs]
docs = [d.replace(".co", "") for d in docs]

# Setting threshold for word frequency - this will determine how nuanced our topics are
min_ct_for_topic = int(len(docs) / 1000) # Change the number in this equation to tune - higher numbers here increases topic ct
print(f"Min Occurrance Count: {min_ct_for_topic}")

Min Occurrance Count: 63


In [ ]:
# Building our Top2Vec model - this cell takes a long time to run!
start = datetime.datetime.now()
tv_model = Top2Vec(docs, 
#                    embedding_model='universal-sentence-encoder', 
                   min_count=min_ct_for_topic, 
                   workers=8, 
                   ngram_vocab=False, 
                   speed="deep-learn")
end = datetime.datetime.now(); elapsed = end-start
print('Cell took a total of {}'.format(elapsed))

In [ ]:
topic_sizes, topic_nums = tv_model.get_topic_sizes()
topic_words, word_scores, topic_nums = tv_model.get_topics()

print("Number of Topics: ",len(topic_nums))
print()
print("First 20 Topic Sizes:", topic_sizes[0:20])

In [ ]:
# Functions for examining our topics and tweet/topic relationships:
def examine_topic(topic, model):
    print('Main Keywords:')
    print(topic_words[topic])
    print('----------')
    print('Keyword Importance:')
    print(word_scores[topic]) # Look at word scores by topic
    print('----------')
    print('Sample Tweets:')
    print('----------')
    documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=topic, num_docs=5)
    for doc, score, doc_id in zip(documents, document_scores, document_ids):
        print(f"### Document: {doc_id}, Score: {score} ###")
        print(doc)
        print('----------')

# Individual Tweet Lookup
def tweet_topic_lookup(tweet_int, model, df=df):
    display(df.iloc[[tweet_int]].style.set_properties(**{'text-align': 'left'})) # Testing document correlation with DF by looking up tweets
    display(model.get_documents_topics([tweet_int])[1]) # Look at top2vec confidence of single tweet
    display(model.get_documents_topics([tweet_int])[2]) # Look at top2vec topic keywords compared to a single tweet

In [ ]:
examine_topic(140, tv_model) # Examine specific topic

In [ ]:
tweet_topic_lookup(61127, tv_model)

In [ ]:
# for loop - look up each document and append the topic ID to the original dataframe
topic_ids = []
for doc in tv_model.document_ids: # model.document_ids # List of all document IDs
    # tv_model.get_documents_topics([doc])[1][0] # Grabbing Confidence
    topic_ids.append(tv_model.get_documents_topics([doc])[0][0]) # Grabbing Topic ID
df['topic'] = topic_ids

In [ ]:
df['topic'].to_csv('data/topics.csv')